# Tutorial 4: Multi-level column indices (`MultiIndex`)

A MultiIndex is an index with multiple, hierarchical levels. We use a multiindex for the column headers of a 
dataframe. This allows each column to have multiple keys associated with it--one key for each level. 

The multiindex is a native part of the `pandas` package. For more documentation, see their [MultiIndex / advanced indexing page](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html).

MultiIndexes make it easy to associate multiple identifiers with a single column.  To show what this looks like we'll look at phosphoproteomics data in the endometrial cancer dataset.

In [1]:
# Throughout the tutorial we will be using the endometrial and colon datasets
import cptac
#cptac.download_cancer('ucec')
en = cptac.Ucec()
en.list_data_sources()

,Data type,Available sources
0,CNV,washu
1,circular_RNA,bcm
2,clinical,mssm
3,deconvolution_cibersort,washu
4,deconvolution_xcell,washu
5,follow-up,mssm
6,medical_history,mssm
7,miRNA,washu
8,phosphoproteomics,umich
9,proteomics,umich


In [2]:

phospho = en.get_proteomics('umich')
en.get_phosphoproteomics('umich')
#en.get_CNV('washu')

Name                     ARF5                  M6PR             ESRRA  \
Site                     S137                  S267               S19   
Peptide       QDMPNAMPVsELTDK GVGDDQLGEEsEERDDHLLPM        AEPAsPDSPK   
Database_ID ENSP00000000233.5     ENSP00000000412.3 ENSP00000000442.6   
Patient_ID                                                              
C3L-00006                 NaN              0.573633               NaN   
C3L-00008            0.003632             -0.393734               NaN   
C3L-00032                 NaN             -0.211020               NaN   
C3L-00084                 NaN              0.220473               NaN   
C3L-00090                 NaN              0.161496               NaN   
...                       ...                   ...               ...   
NX5.N               -0.311416              0.211987               NaN   
NX6.N                0.001346              0.513387               NaN   
NX7.N                0.186926              0.141754               NaN   
NX8.N                0.227515              0.120051               NaN   
NX9.N                0.183929              0.171886               NaN   

Name                                             \
Site                      S22            S19S22   
Peptide            AEPASPDsPK        AEPAsPDsPK   
Database_ID ENSP00000000442.6 ENSP00000000442.6   
Patient_ID                                        
C3L-00006                 NaN          0.304721   
C3L-00008                 NaN          0.789193   
C3L-00032                 NaN               NaN   
C3L-00084                 NaN         -0.290506   
C3L-00090                 NaN          0.708453   
...                       ...               ...   
NX5.N                0.395850          0.045874   
NX6.N               -0.335033         -0.108266   
NX7.N               -0.106618         -0.313644   
NX8.N                     NaN         -0.059253   
NX9.N                     NaN          0.088318   

Name                                                                         \
Site                                    T31                          S19S22   
Peptide     AEPASPDSPKGSSETEtEPPVALAPGPAPTR AEPAsPDsPKGSSETETEPPVALAPGPAPTR   
Database_ID               ENSP00000000442.6               ENSP00000000442.6   
Patient_ID                                                                    
C3L-00006                               NaN                             NaN   
C3L-00008                               NaN                             NaN   
C3L-00032                               NaN                             NaN   
C3L-00084                               NaN                             NaN   
C3L-00090                               NaN                        0.405402   
...                                     ...                             ...   
NX5.N                                   NaN                             NaN   
NX6.N                                   NaN                             NaN   
NX7.N                                   NaN                             NaN   
NX8.N                                   NaN                             NaN   
NX9.N                                   NaN                             NaN   

Name                                                                         \
Site                              S19S22S27                       S19S22T31   
Peptide     AEPAsPDsPKGSsETETEPPVALAPGPAPTR AEPAsPDsPKGSSETEtEPPVALAPGPAPTR   
Database_ID               ENSP00000000442.6               ENSP00000000442.6   
Patient_ID                                                                    
C3L-00006                               NaN                             NaN   
C3L-00008                               NaN                             NaN   
C3L-00032                               NaN                             NaN   
C3L-00084                               NaN                             NaN   
C3L-00090                          1.2

The first level of keys is the name of the gene associated with the column of data. The second level is the site of acetylation. Each column has its own Name and Site associated with it. 
Normally, as above, keys that repeat for adjacent columns are not displayed, to make it easier to read the dataframe. For example, in the table above the first 7 columns are all for the A2M gene, but the label is only shown for the first column. 

# Join functions with multiindices 
The join functions have been written to handle multiindices. More information on the join functions can be found in the joining_dataframes tutorial. 
An example of joining a multiindexed dataframe (in this case phosphoproteomics) with a non multiindexed dataframe (in this case CNV) is below. 

In [3]:
phospho_and_CNV = en.join_omics_to_omics(df1_name="CNV", df2_name="phosphoproteomics", df1_source='washu', df2_source = 'umich')
phospho_and_CNV.head()

DATATYPE: CNV
source: washu
LOAD FUNCTIONS: {'somatic_mutation': <bound method WashuUcec.load_somatic_mutation of <cptac.cancers.washu.washuucec.WashuUcec object at 0x7ff6d4214e80>>, 'miRNA': <bound method WashuUcec.load_miRNA of <cptac.cancers.washu.washuucec.WashuUcec object at 0x7ff6d4214e80>>, 'xcell': <bound method WashuUcec.load_xcell of <cptac.cancers.washu.washuucec.WashuUcec object at 0x7ff6d4214e80>>, 'cibersort': <bound method WashuUcec.load_cibersort of <cptac.cancers.washu.washuucec.WashuUcec object at 0x7ff6d4214e80>>, 'CNV': <bound method WashuUcec.load_CNV of <cptac.cancers.washu.washuucec.WashuUcec object at 0x7ff6d4214e80>>, 'tumor_purity': <bound method WashuUcec.load_tumor_purity of <cptac.cancers.washu.washuucec.WashuUcec object at 0x7ff6d4214e80>>}


Polars found a filename. Ensure you pass a path to the file instead of a python file object when possible for best performance.

DATATYPE: phosphoproteomics
source: umich
LOAD FUNCTIONS: {'phosphoproteomics': <bound method UmichUcec.load_phosphoproteomics of <cptac.cancers.umich.umichucec.UmichUcec object at 0x7ff6d4214ca0>>, 'proteomics': <bound method UmichUcec.load_proteomics of <cptac.cancers.umich.umichucec.UmichUcec object at 0x7ff6d4214ca0>>}


Name       A1BG_washu_CNV A1CF_washu_CNV A2M_washu_CNV A2ML1_washu_CNV  \
Site                                                                     
Peptide                                                                  
Patient_ID                                                               
C3L-00006        -0.00659       -0.01982      -0.01402        -0.01402   
C3L-00008         0.02578        0.00726       0.01350         0.01350   
C3L-00032         0.01262        0.00425      -0.00275        -0.00275   
C3L-00084             NaN            NaN           NaN             NaN   
C3L-00090         0.00100        0.41191      -0.02299        -0.02299   

Name       A3GALT2_washu_CNV A4GALT_washu_CNV A4GNT_washu_CNV AAAS_washu_CNV  \
Site                                                                           
Peptide                                                                        
Patient_ID                                                                     
C3L-00006           -0.01418         -0.00839        -0.01305       -0.01402   
C3L-00008            0.00732          0.01642         0.01005        0.01225   
C3L-00032            0.00166          0.00549        -0.00038       -0.00275   
C3L-00084                NaN              NaN             NaN            NaN   
C3L-00090           -0.02436         -0.01198        -0.03307       -0.02299   

Name       AACS_washu_CNV AADAC_washu_CNV  ...  \
Site                                       ...   
Peptide                                    ...   
Patient_ID                                 ...   
C3L-00006        -0.01402        -0.01305  ...   
C3L-00008         0.01225         0.01005  ...   
C3L-00032        -0.00275        -0.00038  ...   
C3L-00084             NaN             NaN  ...   
C3L-00090        -0.02299        -0.03307  ...   

Name                 SCRIB_umich_phosphoproteomics             \
Site                               S1575T1588S1594      S1594   
Peptide    LAEAPSPAPTPsPTPVEDLGPQTStSPGRLsPDFAEELR LsPDFAEELR   
Patient_ID                                                      
C3L-00006                                      NaN   0.667426   
C3L-00008                                      NaN        NaN   
C3L-00032                                      NaN   0.104862   
C3L-00084                                      NaN   0.399718   
C3L-00090                                      NaN   1.069439   

Name       TSGA10_umich_phosphoproteomics                                  \
Site                                  S11       S173      S213       S391   
Peptide                sPGRDPELQVEAAEVTTK sPSRLDSFVK   RPsPTAR AMDTEsELGR   
Patient_ID                                                                  
C3L-00006                             NaN   0.905606       NaN  -0.069911   
C3L-00008                             NaN  -0.488427       NaN        NaN   
C3L-00032                             NaN        NaN       NaN        NaN   
C3L-00084                       -0.875016  -0.579824       NaN        NaN   
C3L-00090                             NaN   0.510268 -1.889144        NaN   

Name                                                          \
Site                                S779                S101   
Peptide    GLDRsLEENLCYR;GLDRsLEENLCYRDF EVVSSQVDDLTsHNEHLCK   
Patient_ID                                                     
C3L-00006                      -0.584774                 NaN   
C3L-00008                            NaN                 NaN   
C3L-00032                            NaN                 NaN   
C3L-00084                      -0.505807                 NaN   
C3L-00090                      -0.592203                 NaN   

Name       SVIL_umich_phosphoproteomics            
Site                               S296      S459  
Peptide                DSEGDTPsLINWPSSK  LPsPTVAR  
Patient_ID                                         
C3L-00006                     -0.561657 -0.652457  
C3L-00008                     -0.431599 -1.0

Since the CNV dataframe doesn't have the level "Site", it is filled in with NANs, so that it can be joined to the phosphoproteomics dataframe. 

# How to select from multiindex


## Selecting based on all levels
We can select single columns by passing the proper keys for all levels of the multiindex. For example, to get the acetylproteomics for A2M at site K1176, we'd do the following:

In [4]:
acetyl = en.get_acetylproteomics()
all_levels_selection = acetyl["A2M"]["K1176"]
all_levels_selection.head(10)

AttributeError: 'Ucec' object has no attribute 'get_acetylproteomics'

## Selecting based on one level
We can easily select multiple columns from our multiindex dataframe, based on just the "Name" level of the multiindex:

In [ ]:
gene1_filter = acetyl.columns.get_level_values("Name").str.startswith("AA") # Select all columns where the gene starts with "AA". This will grab every column where the key "Name" starts with AA
gene1_data = acetyl.loc[:, gene1_filter]
gene1_data.head()

## Selecting based on a different level of the multiindex
We can also select based on one of the inner levels of the multiindex. For example, to get data for all tyrosine phosphorylation sites:

In [ ]:
y_site_filter = phospho.columns.get_level_values("Site").str.contains("Y") # Create a boolean filter selecting all columns where the Site level contains a "Y"

y_sites = phospho.loc[:, y_site_filter] # Select the columns
y_sites.head()

# How to use `cptac.utils.reduce_multiindex()`
To make it easier to work with multi-level indices, we provide the `reduce_multiindex` function, available for import from the `cptac.utils` submodule. It can both drop levels from a multiindex, and "flatten" a multi-level index into a single-level index by concatenating the keys from multiple levels into a single key for each column.

In [ ]:
import cptac.utils as ut

## Dropping Levels
We can drop levels based on index or name. We can also drop single or multiple levels at once. 
Note that it will warn you if duplicate column key combinations arise due to dropping levels. 

### Dropping by index or name

In [ ]:
ut.reduce_multiindex(df=phospho, levels_to_drop="Site").head()

### Dropping single or multiple levels at once
By passing a list (or array-like) to levels_to drop, we can drop multiple levels of the multiindex at the same time. Note that we must leave at least one existing level. 

We will show this with the colon data.

In [ ]:
colon = cptac.Colon()
phospho = colon.get_phosphoproteomics()
phospho.head()

In [ ]:
# Drop evey level except Database_ID
drop = ["Name", "Site"]
ut.reduce_multiindex(df=phospho, levels_to_drop=drop).head()

## Combining levels (Flattening)

We can combine levels of a multiindexed dataframe. When combined the levels will be sepereated by an underscore, by default. We could specify a different seperator using the `sep` parameter.

In [ ]:
ut.reduce_multiindex(df=phospho, flatten=True).head()

When flatteing levels , NaNs and empty strings will automitically be dropped.

In [ ]:
phospho_and_CNV = en.join_omics_to_omics(df1_name="CNV", df2_name="phosphoproteomics")
phospho_and_CNV.head()

# Note that the CNV columns all have empty strings in the "Site" level of the columns,
# since the CNV data doesn't have any values for that.

In [ ]:
ut.reduce_multiindex(df=phospho_and_CNV, flatten=True).head()
# Notice that the empty strings have been dropped

## Getting a single level index of tuples

You can also use `reduce_multiindex` to turn the multi-level column index into a single level index of tuples, with each value in a column's tuple corresponding to the column's value for that level of the index:

In [ ]:
ut.reduce_multiindex(df=phospho, tuples=True).head()

## Turning off warnings

If your multiindex operation creates duplicate column headers, or has no effect, `reduce_multiindex` will warn you. You can silence these warnings by passing `True` to the `quiet` parameter:

In [ ]:
ut.reduce_multiindex(df=phospho, levels_to_drop=["Name", "Database_ID"]).head()

In [ ]:
# No warning will be issued
ut.reduce_multiindex(df=phospho, levels_to_drop=["Name", "Database_ID"], quiet=True).head()